<h1 align=center><font size = 5>BATTLE OF NEIGHBORHOODS IN MADRID</font></h1>

First import the libraries that will be needed.

In [2]:
import pandas as pd  # library for data analsysis
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import numpy as np  # library to handle data in a vectorized manner

import json # library to handle JSON files

!conda install -c conda-forge wikipedia --yes
import wikipedia as wp

import requests # library to handle requests
#####from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

!conda install -c conda-forge geopy --yes
from geopy.distance import geodesic 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - wikipedia


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    wikipedia-1.4.0            |             py_2          13 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    wikipedia:       1.4.0-py_2        conda-forge

The following packages will be UPDATED:

    ca-certificates: 2019.11.27-0                  --> 2019.11.28-hecc5488_0 conda-forge
    certifi:         2019.11.28-py36_0     

## 1. Import and process data

Improt the list of districts and neighborhoods of Madrid from Wikipedia.

In [3]:
html = wp.page("Anexo:Distritos de Madrid").html()
df = pd.read_html(html)[1]

# Drop variables that are not of interest for our problem
df.drop(columns=['District Number','District area[n 1] (Ha.)','Population','Population density(Hab./Ha.)','Location'], inplace = True)
df.drop(df.index[df['Name'].count()-1], inplace = True)
df.rename(columns={'Name':'District','Administrative wards':'Neighborhood'}, inplace = True)

print('There are {} districts in Madrid.'.format(df['District'].count()))
df.head()

There are 21 districts in Madrid.


,District,Neighborhood
0,Centro,Palacio (11)Embajadores (12)Cortes (13)Justici...
1,Arganzuela,Imperial (21)Acacias (22)Chopera (23)Legazpi (...
2,Retiro,Pacífico (31)Adelfas (32)Estrella (33)Ibiza (3...
3,Salamanca,Recoletos (41)Goya (42)Fuente del Berro (43)Gu...
4,Chamartín,El Viso (51)Prosperidad (52)Ciudad Jardín (53)...


The neighborhoods within each district are all on the same row, sepparated by ',' and and ID number in braquets. Neighbourhoods need to be sepparated and the ID part removed. Also, get the geographical coordinates of each neighborhood using geocoder. Finally, find distance from each neighbourhood to the Spanish National Research Center, called 'Consejo Superior de Investigaciones Cientificas sede central'. Put all this data into a new dataframe.

In [4]:
# initialize variables
neighborhoods = []   # Neighborhood names
districts = []   # District to which each neighborhood belongs
Latitude = []   # Latitude coordinate of each neighborhood
Longitude = []   # Longitude coordinate of each neighborhood
distance = []   # Distance from each neighborhood to the origin research center

# Find coordinates of the research center
g = geocoder.arcgis('Calle de Serrano, 117, 28010 Madrid, Spain')
lat_lng_coords_RC = g.latlng
#latitude_RC = lat_lng_coords_RC[0]
#longitude_RC = lat_lng_coords_RC[1]

for i in range(1,df['District'].count()):
    dis = df['District'][i]
    neigs = df['Neighborhood'][i]
    a = neigs.split(')')
    a = a[0:len(a)-1]
    
    for n in a:
        b = n[0:n.find(' (')]
        neighborhoods.append(b)
        districts.append(dis)
        
        # loop until you get the coordinates
        lat_lng_coords = None
        while(lat_lng_coords is None):
            address = '{}, Madrid'.format(b)
            g = geocoder.arcgis(address)
            lat_lng_coords = g.latlng
            
        Latitude.append(lat_lng_coords[0])
        Longitude.append(lat_lng_coords[1])
        
        # Compute distance from research center in km        
        dist = geodesic(lat_lng_coords_RC, lat_lng_coords).km
        distance.append(dist)

print('There are {} neighborhoods in Madrid.'.format(len(neighborhoods)))
        
data = pd.DataFrame({'Neighborhood':neighborhoods, 'District':districts, 'Latitude':Latitude, 'Longitude':Longitude, 'Distance':distance})
data.head()

There are 125 neighborhoods in Madrid.


,Neighborhood,District,Latitude,Longitude,Distance
0,Imperial,Arganzuela,40.40833,-3.71865,4.474576
1,Acacias,Arganzuela,40.40137,-3.70669,4.692252
2,Chopera,Arganzuela,40.39536,-3.69833,5.159600
3,Legazpi,Arganzuela,40.38702,-3.68990,6.013740
4,Delicias,Arganzuela,40.39613,-3.68946,5.001532


Visualize the city of Madrid and its neighborhoods

In [5]:
# Get latitude and longitude coordinates of Madrid
g = geocoder.arcgis('Madrid, Spain')
lat_lng_coords = g.latlng
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print('The geograpical coordinates of Madrid are {}, {}.'.format(latitude, longitude))

# create map of Madrid using latitude and longitude values
map_Madrid = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(data['Latitude'], data['Longitude'], data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Madrid)  
    
#Add marker at the Spanish National Research Center headquarters
folium.CircleMarker(
    lat_lng_coords_RC,
    radius=5,
    popup='Spanish National Research Center Headquarters',
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Madrid)
    
map_Madrid

The geograpical coordinates of Madrid are 40.41955000000007, -3.6919599999999377.


## 2. Explore neighborhoods in Madrid

Define Foursquare Credentials and Version

In [6]:
CLIENT_ID = '5TQ1R1DIHISONMOHVOCIIBKYC2UCGXLRW0JD5G30JOEYFMI0' # your Foursquare ID
CLIENT_SECRET = 'QY1QBUEMUO0KWTVCUC0OZQ2BBYQ32M00RDY2HZPWWBIDCIUJ' # your Foursquare Secret
VERSION = '20200127' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5TQ1R1DIHISONMOHVOCIIBKYC2UCGXLRW0JD5G30JOEYFMI0
CLIENT_SECRET:QY1QBUEMUO0KWTVCUC0OZQ2BBYQ32M00RDY2HZPWWBIDCIUJ


Get the first 100 venues within a radius of 500m for each entry in the dataframe.

In [9]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # assign relevant part of JSON to venues
        #dataframe = json_normalize(venues)	# tranform venues into a dataframe

        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,  
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

madrid_venues = getNearbyVenues(names=data['Neighborhood'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )
madrid_venues.head()

,Neighborhood,Venue,Venue Category
0,Imperial,Madrid Río (Sector Norte),Park
1,Imperial,Seoul,Korean Restaurant
2,Imperial,Parque de Atenas,Park
3,Imperial,El Landó,Spanish Restaurant
4,Imperial,Spa Los Nogales,Spa


See the amount of venues in each category in each of the neighborhoods of Madrid.

In [10]:
madrid_venues.groupby(['Neighborhood','Venue Category']).count()

Venue
Neighborhood    Venue Category                 
Abrantes        Athletics & Sports            1
                Bakery                        1
                Fast Food Restaurant          1
                Park                          1
                Pizza Place                   1
                Plaza                         1
Acacias         Asian Restaurant              1
                Bar                           4
                Bookstore                     1
                Breakfast Spot                1
                Brewery                       1
                Café                          2
                Chinese Restaurant            1
                Food & Drink Shop             1
                Gym                           2
                Gym / Fitness Center          2
                Hotel                         1
                Ice Cream Shop                1
                Indie Movie Theater           1
                Indie Theater                 2
                Italian Restaurant            1
                Mediterranean Restaurant      1
                Metro Station                 1
                Nightclub                     1
                Park                          2
                Peruvian Restaurant           1
                Pizza Place                   3
                Playground                    1
                Pool                          1
                Portuguese Restaurant         1
...                                         ...
Villaverde Alto Spanish Restaurant            2
                Thrift / Vintage Store        1
                Train Station                 3
Vinateros       Flower Shop                   1
                Food & Drink Shop             1
                Fried Chicken Joint           1
                Gym / Fitness Center          1
                Metro Station                 1
                Playground                    1
                Plaza                         1
                Restaurant                    1
                Shopping Mall                 1
                Skating Rink                  1
Vista Alegre    Athletics & Sports            1
                Bakery                        1
                Comedy Club                   1
                Concert Hall                  1
                Fast Food Restaurant          1
                Indie Theater                 1
                Park                          1
                Pizza Place                   2
                Plaza                         1
                Seafood Restaurant            1
                Spanish Restaurant            1
                Tapas Restaurant              1
Zofío           Asian Restaurant              1
                Athletics & Sports            1
                Beer Garden                   1
                Park                          1
                Spanish Restaurant            3

[1931 rows x 1 columns]

Find out how many unique categories can be curated from all the returned venues

In [11]:
print('There are {} uniques categories.'.format(len(madrid_venues['Venue Category'].unique())))

There are 256 uniques categories.


Group the venue categories that are relevant for our prolem. Por example, rename food cateogires such as 'Spanish Restaurant', 'Tapas Restaurant', 'Vegetarian Restaurant', etc. as just 'Restaurant'

In [12]:
madrid_venues['Group'] = 'Other'

# Group all stores
pos = madrid_venues['Venue Category'].str.contains('Shop') | madrid_venues['Venue Category'].str.contains('Store') | madrid_venues['Venue Category'].str.contains('Bookstore')
pos = pos + madrid_venues['Venue Category'].str.contains('Boutique') | madrid_venues['Venue Category'].str.contains('Pharmacy') | madrid_venues['Venue Category'].str.contains('Flea Market')
madrid_venues.at[pos, 'Group'] = 'Store'

# Group all restaurants
pos = madrid_venues['Venue Category'].str.contains('Restaurant') | madrid_venues['Venue Category'].str.contains('Snack Place') | madrid_venues['Venue Category'].str.contains('Burger')
pos = pos + madrid_venues['Venue Category'].str.contains('Diner') | madrid_venues['Venue Category'].str.contains('Bistro') | madrid_venues['Venue Category'].str.contains('Pizza')
pos = pos + madrid_venues['Venue Category'].str.contains('Breakfast Spot') | madrid_venues['Venue Category'].str.contains('Food') | madrid_venues['Venue Category'].str.contains('Chicekn')
pos = pos + madrid_venues['Venue Category'].str.contains('BBQ') | madrid_venues['Venue Category'].str.contains('Gastropub') | madrid_venues['Venue Category'].str.contains('Taco Place')
pos = pos + madrid_venues['Venue Category'].str.contains('Salad') | madrid_venues['Venue Category'].str.contains('Burrito') | madrid_venues['Venue Category'].str.contains('Bagel')
pos = pos + madrid_venues['Venue Category'].str.contains('Steakhouse') | madrid_venues['Venue Category'].str.contains('Fish & Chips') | madrid_venues['Venue Category'].str.contains('Hot Dog')
pos = pos + madrid_venues['Venue Category'].str.contains('Noodle') | madrid_venues['Venue Category'].str.contains('Frozen Yoghurt') | madrid_venues['Venue Category'].str.contains('Pie')
pos = pos + madrid_venues['Venue Category'].str.contains('Donut') | madrid_venues['Venue Category'].str.contains('Ice Cream') | madrid_venues['Venue Category'].str.contains('Dessert')
pos = pos + madrid_venues['Venue Category'].str.contains('Cupcake') | madrid_venues['Venue Category'].str.contains('Pastry') | madrid_venues['Venue Category'].str.contains('Smoothie')
madrid_venues.at[pos, 'Group'] = 'Restaurant'

# Group all fitness centers
pos = madrid_venues['Venue Category'].str.contains('Gym') | madrid_venues['Venue Category'].str.contains('Yoga Studio') | madrid_venues['Venue Category'].str.contains('Soccer Field')
pos = pos + madrid_venues['Venue Category'].str.contains('Sports Club') | madrid_venues['Venue Category'].str.contains('Athletics & Sports') | madrid_venues['Venue Category'].str.contains('Martial Arts')
pos = pos + madrid_venues['Venue Category'].str.contains('Tennis Court') | madrid_venues['Venue Category'].str.contains('Dance Studio') | madrid_venues['Venue Category'].str.contains('Basketball Court')
pos = pos + madrid_venues['Venue Category'].str.contains('Golf Course') | madrid_venues['Venue Category'].str.contains('Fitness')
madrid_venues.at[pos, 'Group'] = 'Fitness Center'

# Group all grocery stores
pos = madrid_venues['Venue Category'].str.contains('Supermarket') | madrid_venues['Venue Category'].str.contains('Food and Drink Shop') | madrid_venues['Venue Category'].str.contains('Farmers Market')
pos = pos + madrid_venues['Venue Category'].str.contains('Bakery') | madrid_venues['Venue Category'].str.contains('Gourmet Shop') | madrid_venues['Venue Category'].str.contains('Food Store')
pos = pos + madrid_venues['Venue Category'].str.contains('Deli') | madrid_venues['Venue Category'].str.contains('Wine Shop') | madrid_venues['Venue Category'].str.contains('Cheese Shop')
pos = pos + madrid_venues['Venue Category'].str.contains('Fish Market') | madrid_venues['Venue Category'].str.contains('Gourmet Shop') | madrid_venues['Venue Category'].str.contains('Food Store')
pos = pos + madrid_venues['Venue Category'].str.contains('Grocery Store')
madrid_venues.at[pos, 'Group'] = 'Grocery Store'

# Group all coffee & tea shops
pos = madrid_venues['Venue Category'].str.contains('Café') | madrid_venues['Venue Category'].str.contains('Cafeteria') | madrid_venues['Venue Category'].str.contains('Tea') | madrid_venues['Venue Category'].str.contains('Coffee') 
madrid_venues.at[pos, 'Group'] = 'Coffee & Tea'

# Group all types of hotels
pos = madrid_venues['Venue Category'].str.contains('Bed & Breakfast') | madrid_venues['Venue Category'].str.contains('Hostel') | madrid_venues['Venue Category'].str.contains('Hotel')
pos = pos + madrid_venues['Venue Category'].str.contains('Boarding House') | madrid_venues['Venue Category'].str.contains('Resort')
madrid_venues.at[pos, 'Group'] = 'Hotel'

# Group all public transportation
pos = madrid_venues['Venue Category'].str.contains('Station') | madrid_venues['Venue Category'].str.contains('Bus') | madrid_venues['Venue Category'].str.contains('Airport')
madrid_venues.at[pos, 'Group'] = 'Transport'

madrid_venues.groupby(['Neighborhood','Group']).count()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/computation/expressions.py:183: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


Venue  Venue Category
Neighborhood     Group                                
Abrantes         Fitness Center      1               1
                 Grocery Store       1               1
                 Other               2               2
                 Restaurant          2               2
Acacias          Coffee & Tea        2               2
                 Fitness Center      4               4
                 Grocery Store       3               3
                 Hotel               1               1
                 Other              17              17
                 Restaurant         24              24
                 Store               2               2
                 Transport           2               2
Adelfas          Coffee & Tea        3               3
                 Fitness Center      4               4
                 Grocery Store       9               9
                 Hotel               1               1
                 Other              10              10
                 Restaurant         18              18
                 Store               4               4
                 Transport           3               3
Alameda de Osuna Fitness Center      1               1
                 Grocery Store       2               2
                 Other               6               6
                 Restaurant          9               9
                 Store               3               3
                 Transport           1               1
Almagro          Coffee & Tea        4               4
                 Fitness Center      1               1
                 Grocery Store       4               4
                 Hotel               6               6
...                                ...             ...
Vallehermoso     Fitness Center      1               1
                 Grocery Store       3               3
                 Other              11              11
                 Restaurant         20              20
                 Store               1               1
Valverde         Restaurant          3               3
                 Transport           2               2
Ventas           Fitness Center      2               2
                 Grocery Store       4               4
                 Other               3               3
                 Restaurant          5               5
                 Store               1               1
                 Transport           1               1
Villaverde Alto  Grocery Store       1               1
                 Other               1               1
                 Restaurant          4               4
                 Store               2               2
                 Transport           3               3
Vinateros        Fitness Center      1               1
                 Other               4               4
                 Restaurant          2               2
                 Store               2               2
                 Transport           1               1
Vista Alegre     Fitness Center      1               1
                 Grocery Store       1               1
                 Other               5               5
                 Restaurant          6               6
Zofío            Fitness Center      1               1
                 Other               2               2
                 Restaurant          4               4

[597 rows x 2 columns]

Get the amount of venues of each type for each neighborhood

In [33]:
# one hot encoding
madrid_onehot = pd.get_dummies(madrid_venues[['Group']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
madrid_onehot['Neighborhood'] = madrid_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [madrid_onehot.columns[-1]] + list(madrid_onehot.columns[:-1])
madrid_onehot = madrid_onehot[fixed_columns]

# Group rows by neighborhood and compute the amount of venues per type
madrid_grouped = madrid_onehot.groupby('Neighborhood').sum().reset_index()
madrid_grouped.drop(columns='Other', inplace = True)
madrid_grouped.head()

,Neighborhood,Coffee & Tea,Fitness Center,Grocery Store,Hotel,Restaurant,Store,Transport
0,Abrantes,0,1,1,0,2,0,0
1,Acacias,2,4,3,1,24,2,2
2,Adelfas,3,4,9,1,18,4,3
3,Alameda de Osuna,0,1,2,0,9,3,1
4,Almagro,4,1,4,6,57,7,0


Add a column to the dataframe with the distance from the neighborhood to the research center, and sort the neighborhoods by this distance.

In [34]:
madrid_grouped.sort_values(by=['Neighborhood'], inplace=True)
data.sort_values(by=['Neighborhood'], inplace=True)
data = data.reset_index(drop=True)

pos = data['Neighborhood'].isin(madrid_grouped['Neighborhood'])
madrid_grouped['Distance'] = data['Distance'][data['Neighborhood'].isin(madrid_grouped['Neighborhood'])].reset_index(drop=True)
madrid_grouped['Distance'] = madrid_grouped['Distance'].round(2)
madrid_grouped.sort_values(by=['Distance'], inplace=True)

madrid_grouped

,Neighborhood,Coffee & Tea,Fitness Center,Grocery Store,Hotel,Restaurant,Store,Transport,Distance
95,Ríos Rosas,6,1,10,0,34,7,1,0.72
42,El Viso,2,0,2,0,10,0,0,0.73
26,Castellana,10,1,3,4,65,4,0,0.80
4,Almagro,4,1,4,6,57,7,0,0.96
106,Trafalgar,7,1,6,0,55,6,0,1.36
61,Lista,5,1,4,3,49,1,0,1.48
37,Cuatro Caminos,4,2,12,2,27,3,0,1.53
88,Prosperidad,6,2,9,4,20,2,0,1.60
92,Recoletos,2,1,5,8,52,25,0,1.77
10,Arapiles,10,1,10,2,40,7,0,1.96


Keep only the neighborhoods that have at least one of each category

In [35]:
columns = madrid_grouped.columns
temp = madrid_grouped

for i in range(1,len(columns)-1):
    temp = temp[temp[columns[i]] > 0]

temp = temp.reset_index(drop=True)
temp

,Neighborhood,Coffee & Tea,Fitness Center,Grocery Store,Hotel,Restaurant,Store,Transport,Distance
0,Bellas Vistas,2,1,10,1,20,5,2,2.10
1,San Juan Bautista,1,1,5,1,17,4,2,2.96
2,Palos de Moguer,3,3,8,2,38,3,2,4.26
3,Adelfas,3,4,9,1,18,4,3,4.56
4,Acacias,2,4,3,1,24,2,2,4.69


Out of the 125 neighborhoods in Madrid, only 5 of them meet the requirements for our business problem.